In [39]:
!pip install pandas matplotlib openai python-dotenv tabulate -q


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd

In [12]:
df_amazon = pd.read_csv("data/Amazon Sale Report.csv")
df_amazon.head()

/var/folders/22/8544ktfn0gg6g53zkgmhmc9r0000gn/T/ipykernel_58766/3826274105.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_amazon = pd.read_csv("data/Amazon Sale Report.csv")


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [11]:
df_superstore = pd.read_csv("data/Superstore Sales Dataset.csv")
df_superstore.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


# OpenAI

In [21]:
import os
from dotenv import load_dotenv
load_dotenv()
#openai.api_key =  os.getenv('OPENAI_API_KEY')

from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [35]:
def ask_gpt4(input):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input,
            }
        ],
        model="gpt-4-turbo",
    )

    try:
        return chat_completion.choices[0].message.content.strip()
    except Exception as e:
        return str(e)

In [36]:
ask_gpt4("say this is a test")

'This is a test.'

# Matching columns

In [42]:
def df_to_description(df, sample_size=5):
    return df.iloc[1:sample_size].to_markdown()

amazon_table_description = df_to_description(df_amazon)
superstore_table_description = df_to_description(df_superstore)

In [50]:
def reconciliate_table_prompt(tables: list[pd.DataFrame]):
    table_descriptions = []
    for table in tables:
        table_descriptions.append(df_to_description(table))
    
    base_prompt = f"""
        Act as a Data Engineer that need to merge 2 tables from different companies into 1 table. 
        The first step will be to map the columns (or groups of columns) corresponding from 1 table to the other. If some columns cannot be matched, provide in the table them without a matching column

        Provide a csv array where 
            * column 1 = the matching column(s) from table1
            * column 2 = the matching column(s) from table 2.
            * column 3 = boolean indicating that a transformation is needed to make the 2 column format/semantic match

        Table 1 Sample:

        {table_descriptions[0]}

        Table 2 Sample:

        {table_descriptions[1]}


        Output Table:
        ```csv
    """

    return base_prompt

In [51]:
prompt = reconciliate_table_prompt([df_amazon, df_superstore])
output = ask_gpt4(prompt)

In [61]:
output.split('```')[1].replace('csv\n', '')

'Table 1 Column,Table 2 Column,Transformation Needed\nOrder ID,Order ID,False\nDate,Order Date,True\nship-city,City,True\nship-state,State,True\nship-postal-code,Postal Code,False\nship-country,Country,True\nSales Channel,Segment,True\nStyle,Product Name,True\nCategory,Category,False\nSize,Sub-Category,True\nASIN,Product ID,True\nAmount,Sales,True\nIndex,Row ID,True\nship-service-level,Ship Mode,True\nSKU,Product ID,True\nStatus,Order Status,True\nCourier Status,Ship Date,True\nQty,Quantity,False\ncurrency,NA,True\nfulfilled-by,NA,True\nUnnamed: 22,NA,True\npromotion-ids,NA,True\nB2B,NA,True\n'

In [62]:
import io 
pd.read_table(io.StringIO(output.split('```')[1].replace('csv\n', '')), sep=',')

,Table 1 Column,Table 2 Column,Transformation Needed
0,Order ID,Order ID,False
1,Date,Order Date,True
2,ship-city,City,True
3,ship-state,State,True
4,ship-postal-code,Postal Code,False
5,ship-country,Country,True
6,Sales Channel,Segment,True
7,Style,Product Name,True
8,Category,Category,False
9,Size,Sub-Category,True


In [117]:
import random
import numpy as np

# Set the seed to make the results reproducible
random.seed(1337)  # You can choose any number as the seed

def pick_random_ints(sample_size, start, end):
    return random.sample(range(start, end), sample_size)

def describe_column(df, column, max_distinct=10):

    column_properties = df[column].describe(include="all").to_markdown()

    column_distinct = df[column].unique()
    column_distinct = np.array([v if v is not np.nan else "NaN" for v in column_distinct])
    column_distinct_count = len(column_distinct)
    nb_to_sample = min(max_distinct, column_distinct_count)
    column_sample = pick_random_ints(nb_to_sample, 0, column_distinct_count)
    print(column_sample)
    print(column_distinct)
    column_distinct_description = f"Sample values: {';'.join(column_distinct[column_sample])}"

    return f"""
        Column: {column}
        Column Statistics:
        {column_properties}
        {column_distinct_description}
        """


In [92]:
describe_column(df_amazon, "Date")

'\n        Column: Date\n        Column Statistics:\n        |        | Date     |\n|:-------|:---------|\n| count  | 128975   |\n| unique | 91       |\n| top    | 05-03-22 |\n| freq   | 2085     |\n        Sample values: 06-12-22;06-23-22;06-01-22;05-16-22;06-18-22;06-17-22;04-09-22;05-20-22;05-13-22;06-10-22\n        '

In [97]:
def find_column_transform(df1, df2, column1, column2):
    column1_description = describe_column(df1, column1)
    column2_description = describe_column(df2, column2)

    base_prompt = f"""
        Act as a Data Engineer that need to transform 1 column into another. Write the column transformation in pandas.

        Column 1:

        {column1_description}

        Column 2:

        {column2_description}

        Output code:
        ```python
    """

    return base_prompt
    

In [99]:
prompt = find_column_transform(df_amazon, df_superstore, "Date", "Order Date")
output = ask_gpt4(prompt)
output

In [119]:
# Fails on IN which it thinks is India
prompt = find_column_transform(df_amazon, df_superstore, "ship-country", "Country")
print(prompt)
output = ask_gpt4(prompt)
print(output)

[1, 0]
['IN' 'NaN']
[0]
['United States']

        Act as a Data Engineer that need to transform 1 column into another. Write the column transformation in pandas.

        Column 1:

        
        Column: ship-country
        Column Statistics:
        |        | ship-country   |
|:-------|:---------------|
| count  | 128942         |
| unique | 1              |
| top    | IN             |
| freq   | 128942         |
        Sample values: NaN;IN
        

        Column 2:

        
        Column: Country
        Column Statistics:
        |        | Country       |
|:-------|:--------------|
| count  | 9800          |
| unique | 1             |
| top    | United States |
| freq   | 9800          |
        Sample values: United States
        

        Output code:
        ```python
    
```python
import pandas as pd

# Assuming the existing DataFrame is named df

# Transform the data in the 'ship-country' column to the standardized 'Country' column
df['Country'] = df['ship-countr